# MNIST - Neural Network Classification Model

In [16]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import matplotlib.pyplot as plt
%matplotlib inline

## Get data

In [17]:
# grab data from MNIST_data folder
# One hot encode each example
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Tensorflow Graph

#### Placeholders

In [18]:
with tf.name_scope("inputs"):
    x = tf.placeholder(dtype=tf.float32,shape=[None, 784])
    y_true = tf.placeholder(dtype=tf.float32,shape=[None, 10])

#### Variables

In [32]:
# Model Variables
with tf.name_scope("variables"):
    initializer = tf.variance_scaling_initializer()
    w1 = tf.Variable(initializer(shape=[784, 28], dtype=tf.float32))
    b1 = tf.Variable(tf.random_uniform(shape=[28],minval=-1, maxval=1))
    w2 = tf.Variable(initializer(shape=[28, 10], dtype=tf.float32))
    b2 = tf.Variable(tf.random_uniform(shape=[10],minval=-1, maxval=1))

#### Hidden layer

In [33]:
# Hidden layer
with tf.name_scope("hidden_layer"):
    h1 = tf.add(tf.matmul(x, w1), b1)
    z1 = tf.nn.relu(h1)

#### Output

In [34]:
# Output layer
with tf.name_scope("output"):
    z = tf.add(tf.matmul(z1, w2), b2)

#### Cross Entropy Loss

In [35]:
# Cross-Entropy Loss
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=z))

#### Optimizer

In [36]:
# Adam Optimizer
with tf.name_scope("optimizer"):
    optimizer =tf.train.AdamOptimizer(learning_rate=0.0001)
    train = optimizer.minimize(loss)

In [37]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

## Training

In [50]:
num_steps = 15000

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(num_steps):
        
        # Get next batch from training dataset
        batch_x, batch_y = mnist.train.next_batch(100)
        
        # Get loss and accuracy every tenth of our training
        if i % (num_steps / 10) == 0:
            
            # Calculate training and evalutation loss
            train_loss = sess.run(loss, feed_dict={x:mnist.train.images, y_true:mnist.train.labels})
            eval_loss = sess.run(loss, feed_dict={x:mnist.test.images, y_true: mnist.test.labels})
            
            # operations for calculating accuracy
            correct_pred = tf.equal(tf.argmax(z, 1), tf.argmax(y_true, 1))
            acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
            
            # Calculate accuracy on test set
            test_accuracy = sess.run(acc, feed_dict={x: mnist.test.images, y_true:mnist.test.labels})
            
            # Print metrics of current step
            print("step: {:5} train loss: {:1f} eval loss: {:1f}  test_accuracy: {:1f}".format(i, train_loss, eval_loss, test_accuracy))
            
        # Train model on current batch
        sess.run(train, feed_dict={x: batch_x, y_true: batch_y})
    
    print("Finished training")
    
    #Save model
    saver.save(sess, "./models/mnist_nn.ckpt")

step:     0 train loss: 2.654410 eval loss: 2.652552  test_accuracy: 0.113800
step:  1500 train loss: 0.489285 eval loss: 0.469836  test_accuracy: 0.888400
step:  3000 train loss: 0.354108 eval loss: 0.339654  test_accuracy: 0.909100
step:  4500 train loss: 0.309396 eval loss: 0.298879  test_accuracy: 0.917200
step:  6000 train loss: 0.284464 eval loss: 0.277251  test_accuracy: 0.921000
step:  7500 train loss: 0.266990 eval loss: 0.263274  test_accuracy: 0.924700
step:  9000 train loss: 0.253069 eval loss: 0.252451  test_accuracy: 0.926500
step: 10500 train loss: 0.240020 eval loss: 0.241830  test_accuracy: 0.930600
step: 12000 train loss: 0.228755 eval loss: 0.232136  test_accuracy: 0.933600
step: 13500 train loss: 0.218933 eval loss: 0.224562  test_accuracy: 0.935600
Finished training


## Final Test accuracy

In [51]:
with tf.Session() as sess:
    saver.restore(sess, "./models/mnist_nn.ckpt")
    
    correct_pred = tf.equal(tf.argmax(z, 1), tf.argmax(y_true, 1))
    
    acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    print(sess.run(acc, feed_dict={x: mnist.test.images, y_true:mnist.test.labels}))

INFO:tensorflow:Restoring parameters from ./models/mnist_nn.ckpt
0.9386


# ~94 % accuracy